In [1]:
using JuMP
using Gurobi
using StaticArrays
using Rotations
using DataStructures
using GeometryTypes
using DrakeVisualizer
using FileIO
using CoordinateTransformations
using Plots
using PyPlot
using Clp
using ForwardDiff

include("MeshSampling.jl")
using MeshSampling

include("PoseEstimationMathematicalProgram.jl")
using PoseEstimationMathematicalProgram

vis = Visualizer();

In [8]:
# Core parameters we care about

# This many points are sampled on the surface of the model, and
# are used as inliers in the scene cloud.
N_model_points = 20
# This many additional outliers are sampled randomly from a volume around
# the model ([-5, 5]^3) to build the scene cloud.
N_outliers = 5;

In [9]:
box = load("bunny_99_faces.obj")

points = decompose(Point{3,Float64}, box)
faces = decompose(Face{3, Int, 0}, box)

# Sample N points from the surface of the mesh
ret = MeshSampling.samplePointsFromMesh(N_model_points, points, faces);
modelPoints = ret[1]
modelPointsFacesGT = ret[2];

In [10]:
T0 = [0.1; 0; 0.5]
R0 = [0.860089 0.174349 -0.479426; 0.244566 0.683829 0.687434; 0.447698 -0.708506 0.545514]

function convertToPointCloudType(points)
    return PointCloud([[points[1, i], points[2, i], points[3, i]] for i=1:size(points, 2)]) 
end

scenePoints = transpose(R0)*modelPoints + repmat(-transpose(R0)*T0, 1, size(modelPoints, 2))

scenePointsOutliers = (rand(3, N_outliers)-0.5)*1.
scenePoints = cat(2, scenePoints, scenePointsOutliers)
scenePointsFacesGT = cat(1, modelPointsFacesGT, zeros(N_outliers, size(modelPointsFacesGT, 2)))
outliersGTAssignment = zeros(N_model_points+N_outliers, 1)
outliersGTAssignment[N_model_points+1:end] = 1


pointcloud = convertToPointCloudType(scenePoints)
box_vis = setgeometry!(vis[:julia][:box_vis], box)
setgeometry!(vis[:julia][:scene_pts], pointcloud);

In [13]:
function buildModelHelper()
    return PoseEstimationMathematicalProgram.constructModel(
    scenePoints, points, faces, use_relaxed_form = false, rotation_relaxation = :RtRBilinear,
    other_solver=GurobiSolver(TimeLimit = 10000, FeasRelaxBigM = 2, MIPGap = 0.025, Heuristics = 0.1, Threads = 5, OutputFlag=1),
        closeness=10.0, R0_in=R0, optPhiMax=0.1, optBigNumber=10,
    TimeLimit = 3600, FeasRelaxBigM = 20, MIPGap = 0.025, Heuristics = 0.1, Threads = 5, OutputFlag=0);
end
model_data = buildModelHelper();

In [14]:
    print("Solving...\n")

type NodeData
    time::Float64  # in seconds since the epoch
    node::Int
    obj::Float64
    bestbound::Float64
end

bbdata = NodeData[]
last_print = time()-100

function infocallback_intermed(cb)
    node      = MathProgBase.cbgetexplorednodes(cb)
    obj       = MathProgBase.cbgetobj(cb)
    bestbound = MathProgBase.cbgetbestbound(cb)
    push!(bbdata, NodeData(time(),node,obj,bestbound))
    global last_print
    if (time() - last_print > 1.0)
        @printf(" ") #%f: Objective %f, Bound %f\n", time(), obj, bestbound)
        last_print = time()
    end
end

box_fit = setgeometry!(vis[:julia][:box_fit], box)
box_fit_gt = setgeometry!(vis[:julia][:box_fit_gt], box)
tf = AffineMap(R0, T0)
settransform!(box_fit_gt, inv(tf))
last_publish = time() - 100
function infocallback_sol(cb)
    global last_publish
    try
        if time() - last_publish > 1.0
            tf = AffineMap(getvalue(model_data.R), getvalue(model_data.T))
            settransform!(box_fit, inv(tf));
            last_publish = time()
        end
    catch
        ;
    end
end

addinfocallback(model_data.m, infocallback_intermed, when = :Intermediate)
addinfocallback(model_data.m, infocallback_sol, when = :MIPSol)

for i in 1:1
    for k in 1:size(scenePointsFacesGT, 2)
        if (scenePointsFacesGT[i, k] == 1)
            @constraint(model_data.m, model_data.f[i, k] == 1)
        end
    end
end

status = solve(model_data.m)

println("R0: ", R0)
println("T0: ", T0)
println("Objective value: ", getobjectivevalue(model_data.m))
println("Optimal R: ", getvalue(model_data.R))
println("Optimal T: ", getvalue(model_data.T))
println("Optimal phi: ", getvalue(model_data.phi))
println("Optimal alpha: ", getvalue(model_data.alpha))
println("Optimal f: ", getvalue(model_data.f))
println("Optimal fuot: ", getvalue(model_data.f_outlier))
println("Optimal C: ", getvalue(model_data.C))

Solving...
Optimize a model with 2001 rows, 4693 columns and 26609 nonzeros
Variable types: 2049 continuous, 2644 integer (2644 binary)
Coefficient statistics:
  Matrix range     [2e-03, 1e+01]
  Objective range  [4e-03, 4e-02]
  Bounds range     [1e+00, 1e+01]
  RHS range        [1e+00, 1e+00]
Presolve removed 83 rows and 175 columns
Presolve time: 0.06s
Presolved: 1918 rows, 4518 columns, 25182 nonzeros
Variable types: 1974 continuous, 2544 integer (2544 binary)
 


Root relaxation: objective 0.000000e+00, 1089 iterations, 0.05 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0    0.00000    0  161          -    0.00000      -     -    0s
     0     0    0.00000    0  177          -    0.00000      -     -    0s
     0     0    0.00000    0  169          -    0.00000      -     -    0s
     0     0    0.00000    0  182          -    0.00000      -     -    0s
     0     0    0.00000    0  182          -    0.00000      -     -    0s
     0     0    0.00000    0  183          -    0.00000      -     -    0s
H    0     0                      12.4184069    0.00000   100%     -    0s
     0     0    0.00000    0  183   12.41841    0.00000   100%     -    0s
H    0     0                       0.1303444    0.00000   100%     -    1s
      0     2    0.00000    0  183    0.13034    0.00000   100%     -    1s
 H   64    69            

error in running finalizer: TypeError(func=:mastercallback, context="typeassert", expected=Gurobi.GurobiMathProgModel, got=<?#0x7fb633f670d0::<?#0x7fb633f67180::(nil)>>)


  177240 74660    0.01622   45  135    0.02557    0.00812  68.2%   188  586s
    179077 75703    0.02459   44  139    0.02557    0.00812  68.2%   188  590s
    181599 77076    0.01284   31  155    0.02557    0.00813  68.2%   187  595s
     183607 78151    0.01079   30  162    0.02557    0.00814  68.2%   187  600s
     186023 79450     cutoff   43         0.02557    0.00815  68.1%   186  605s
  H188277 80274                       0.0255029    0.00816  68.0%   185  609s
  188513 80347    0.02240   35  168    0.02550    0.00816  68.0%   185  610s
    190678 81429    0.02225   33  157    0.02550    0.00817  68.0%   185  615s
    192756 82578    0.01475   37  153    0.02550    0.00818  67.9%   184  620s
     195312 84027    0.01294   32  164    0.02550    0.00819  67.9%   184  625s
   196878 84609    0.01509   38  153    0.02550    0.00820  67.9%   184  630s
     198994 85658     cutoff   29         0.02550    0.00820  67.8%   184  635s
    200819 86646    0.01304   31  155    0.02550    0.

 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.331715 0.0 0.0 0.303156 0.0 0.365129 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.120479 0.0 0.195136 0.0 0.0 0.684385 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.216488 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.751281 0.0 0.0 0.032231 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.064904 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.543578 0.0 0.0 0.0 0.0 0.0 0.391518 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.

In [15]:
box_fit = setgeometry!(vis[:julia][:box_fit], box)
tf = AffineMap(getvalue(model_data.R), getvalue(model_data.T))
settransform!(box_fit, inv(tf))

box_fit_gt = setgeometry!(vis[:julia][:box_fit_gt], box)
tf = AffineMap(R0, T0)
settransform!(box_fit_gt, inv(tf))

Set{Array{Symbol,1}}()

In [ ]:
optR = getvalue(model_data.R)
println("R^T R: ", transpose(optR) * optR)
println("Det: ", det(optR))